In [1]:
import os
import re
import math
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.cuda.amp as amp
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.optim import lr_scheduler

import tensorflow as tf

from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.8 MB/s eta 0:00:00


In [3]:
from transformers import BertConfig, BertForSequenceClassification, BertTokenizer

In [4]:
!nvidia-smi

Thu Jun 29 13:17:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Model

In [5]:
config = {
    "architectures": [
      "BertForPreTraining"
    ],
    "attention_probs_dropout_prob": 0.1,
    "emb_size": 312,
    "gradient_checkpointing": False,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 312,
    "initializer_range": 0.02,
    "intermediate_size": 600,
    "layer_norm_eps": 1e-12,
    "max_position_embeddings": 2048,
    "model_type": "bert",
    "num_attention_heads": 12,
    "num_hidden_layers": 3,
    "pad_token_id": 0,
    "position_embedding_type": "absolute",
    "torch_dtype": "float32",
    "transformers_version": "4.30.2",
    "type_vocab_size": 2,
    "use_cache": True,
    "vocab_size": 83828
}

In [6]:
model = BertForSequenceClassification(BertConfig(**config))
model.classifier = nn.Linear(312, 13)

model = model.cuda()
model.load_state_dict(torch.load("bert-food-cls.pth"))

model = model.bert
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(83828, 312, padding_idx=0)
    (position_embeddings): Embedding(2048, 312)
    (token_type_embeddings): Embedding(2, 312)
    (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-2): 3 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=312, out_features=312, bias=True)
            (key): Linear(in_features=312, out_features=312, bias=True)
            (value): Linear(in_features=312, out_features=312, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=312, out_features=312, bias=True)
            (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
   

## Data embeddings

In [7]:
tokenizer = BertTokenizer.from_pretrained("cointegrated/rubert-tiny2")

In [8]:
df_items = pd.read_csv("food-dataset-full.csv")

In [9]:
def preproccesing(text, tokenizer):
    """
    Предобработка входного текста
    """
    text = tokenizer(text, add_special_tokens=True, max_length=256,
                  padding='max_length', truncation=True, return_tensors='pt')
    return text

In [10]:
# Получаем векторные представления каждого продукта
items_embed = []
for i in range(df_items.shape[0]):
    text = " ".join(df_items.iloc[i, [0, 2, 3, 4, 5]].values.tolist())
    text = preproccesing(text, tokenizer)

    text['input_ids'] = text['input_ids'].cuda()
    text['token_type_ids'] = text['token_type_ids'].cuda()
    text['attention_mask'] = text['attention_mask'].cuda()

    with torch.no_grad():
        out = model(**text)

    items_embed.append(out.last_hidden_state[:, 0, :].cpu().numpy())

## Recommedation

In [11]:
def get_recommendation(text_tokens, model, items, df_items):
    """
    Вычисления схожих продуктов для рекомендации
    """
    text_tokens['input_ids'] = text_tokens['input_ids'].cuda()
    text_tokens['token_type_ids'] = text_tokens['token_type_ids'].cuda()
    text_tokens['attention_mask'] = text_tokens['attention_mask'].cuda()

    with torch.no_grad():
        out = model(**text_tokens)
        out = out.last_hidden_state[:, 0, :].cpu().numpy()

    rating = []
    for i in range(len(items)):
        rating.append(cosine_similarity(items[i], out).reshape(-1)[0])
    rating_arg = np.argsort(rating)[::-1]

    return rating_arg.tolist()

Первая рекомендация

In [12]:
idx = np.random.randint(df_items.shape[0])
text_input = " ".join(df_items.iloc[idx, [0, 2, 3, 4, 5]].values.tolist())
text_input = preproccesing(text_input, tokenizer)
df_items.iloc[[idx]].head()

,name,text,ingredient,energy,time_cook,type_kitchen,label
34529,Разноцветные американские кукис,Разогрейте духовку до 180 градусов. Сливочное ...,"Сливочное масло: 120 г, Ванилин: 1 чайная ложк...","Калорий 2672 ккал, Белки 32 грамм, Жиры 134 гр...",Американская кухня,1 час,Выпечка


In [13]:
rating = get_recommendation(text_input, model, items_embed, df_items)
df_items.iloc[rating].head()

,name,text,ingredient,energy,time_cook,type_kitchen,label
34529,Разноцветные американские кукис,Разогрейте духовку до 180 градусов. Сливочное ...,"Сливочное масло: 120 г, Ванилин: 1 чайная ложк...","Калорий 2672 ккал, Белки 32 грамм, Жиры 134 гр...",Американская кухня,1 час,Выпечка
33568,Кукисы с кусочками шоколада и М&М’s,Размягченное масло взбить с сахаром до белой п...,"Сливочное масло: 150 г, Тростниковый сахар: 1,...","Калорий 366 ккал, Белки 5 грамм, Жиры 15 грамм...",Армянская кухня,1 час,Выпечка
30062,Сметанный пирог с малиной,Яйца с сахаром взбить миксером на большой скор...,"Сметана: 200 г, Сахар: 250 г, Пшеничная мука: ...","Калорий 599 ккал, Белки 11 грамм, Жиры 24 грам...",Европейская кухня,2 часа,Выпечка
32003,Ананасовый пирог-перевертыш,Разогреть духовку до 180 градусов. В чаше микс...,"Сливочное масло: 230 г, Мед: 1,5 столовых ложе...","Калорий 768 ккал, Белки 7 грамм, Жиры 35 грамм...",Европейская кухня,1 час,Выпечка
32536,Капкейки с шоколадной глазурью,Сахар и ванильный сахар растереть с маслом. До...,"Куриное яйцо: 2 штуки, Кефир: 200 мл, Пшенична...","Калорий 218 ккал, Белки 5 грамм, Жиры 9 грамм,...",Американская кухня,1 час,Выпечка


Вторая рекомендация

In [14]:
idx = np.random.randint(df_items.shape[0])
text_input = " ".join(df_items.iloc[idx, [0, 2, 3, 4, 5]].values.tolist())
text_input = preproccesing(text_input, tokenizer)
df_items.iloc[[idx]].head()

,name,text,ingredient,energy,time_cook,type_kitchen,label
12621,Говядина в пяти специях,"Куриные и свиные кости залить кипятком (5 л), ...","Куриные кости : 1 кг, Свиные кости: 1 кг, Имби...","Калорий 1178 ккал, Белки 68 грамм, Жиры 82 гра...",Европейская кухня,45 минут,Основное блюдо


In [15]:
rating = get_recommendation(text_input, model, items_embed, df_items)
df_items.iloc[rating].head()

,name,text,ingredient,energy,time_cook,type_kitchen,label
12621,Говядина в пяти специях,"Куриные и свиные кости залить кипятком (5 л), ...","Куриные кости : 1 кг, Свиные кости: 1 кг, Имби...","Калорий 1178 ккал, Белки 68 грамм, Жиры 82 гра...",Европейская кухня,45 минут,Основное блюдо
13919,Говядина по-японски,"Говядину хорошо промыть, положить в ёмкость, з...","Говядина без кости: 1 кг, Капуста: 1 кг, Репча...","Калорий 736 ккал, Белки 56 грамм, Жиры 46 грам...",Японская кухня,1 час 10 минут,Основное блюдо
17149,Говядина с грецкими орехами и грибами,Говядину отварить в под солёной воде до готовн...,"Говядина: 1 кг, Грецкие орехи: 100 г, Грибы: 3...","Калорий 624 ккал, Белки 44 грамм, Жиры 48 грам...",Русская кухня,1 час 10 минут,Основное блюдо
10260,Говядина в томатном соусе с фасолью,Нарежьте говядину небольшими кусочками. Обжарь...,"Говядина: 700 г, Помидоры: 3 штуки, Консервиро...","Калорий 655 ккал, Белки 41 грамм, Жиры 45 грам...",Мексиканская кухня,1 час,Основное блюдо
14534,Говядина с картофелем в рукаве,"Картофель почистить, промыть под проточной вод...","Говядина: 500 г, Картофель: 500 г, Морковь: 1 ...","Калорий 414 ккал, Белки 28 грамм, Жиры 21 грам...",Русская кухня,1 час,Основное блюдо


Третья рекомендация

In [16]:
idx = np.random.randint(df_items.shape[0])
text_input = " ".join(df_items.iloc[idx, [0, 2, 3, 4, 5]].values.tolist())
text_input = preproccesing(text_input, tokenizer)
df_items.iloc[[idx]].head()

,name,text,ingredient,energy,time_cook,type_kitchen,label
6177,Форшмак «Дружба народов»,"Почистить селедку, или купить готовое филе в м...","Морковь: 1 штука, Плавленый сырок: 3 штуки, Фи...","Калорий 277 ккал, Белки 21 грамм, Жиры 14 грам...",Европейская кухня,20 минут,Закуска


In [17]:
rating = get_recommendation(text_input, model, items_embed, df_items)
df_items.iloc[rating].head()

,name,text,ingredient,energy,time_cook,type_kitchen,label
6177,Форшмак «Дружба народов»,"Почистить селедку, или купить готовое филе в м...","Морковь: 1 штука, Плавленый сырок: 3 штуки, Фи...","Калорий 277 ккал, Белки 21 грамм, Жиры 14 грам...",Европейская кухня,20 минут,Закуска
2523,Форшмак,Отрезать от батона — можно обычного нарезного ...,"Филе сельди: 500 г, Куриное яйцо: 3 штуки, Бел...","Калорий 227 ккал, Белки 18 грамм, Жиры 13 грам...",Еврейская кухня,15 минут,Закуска
3517,Форшмак классический,Сварить яйца вкрутую. Сельдь нарезать небольши...,"Филе соленой сельди: 400 г, Репчатый лук: 100 ...","Калорий 448 ккал, Белки 21 грамм, Жиры 37 грам...",Еврейская кухня,30 минут,Закуска
7296,Форшмак классический,Филе сельди вымочить в стакане молока в течени...,"Филе сельди: 4 штуки, Молоко: 225 мл, Яблоко: ...","Калорий 432 ккал, Белки 29 грамм, Жиры 27 грам...",Еврейская кухня,30 минут,Закуска
3796,Форшмак Лары Кацовой,Мелко нарезать лук-шалот. Снять корки со сверд...,"Слабосоленая сельдь: 2 штуки, Сливочное масло:...","Калорий 485 ккал, Белки 204 грамм, Жиры 37 гра...",Европейская кухня,15 минут,Закуска


Четвертая рекомендация

In [27]:
idx = np.random.randint(df_items.shape[0])
text_input = "Ризотто"
print(text_input)
text_input = preproccesing(text_input, tokenizer)

Ризотто


In [28]:
rating = get_recommendation(text_input, model, items_embed, df_items)
df_items.iloc[rating].head()

,name,text,ingredient,energy,time_cook,type_kitchen,label
20093,Ризотто с мускатной тыквой,Обжарьте лук с 1 ст. л. до мягкости (не допуск...,"Репчатый лук: 1 головка, Оливковое масло: по в...","Калорий 326 ккал, Белки 12 грамм, Жиры 6 грамм...",Европейская кухня,45 минут,Ризотто
20078,Ризотто с осьминогом (Risotto con polpo),Мясо осьминога промыть и варить в кипящей воде...,"Осьминог: 1 кг, Рис: 500 г, Белое сухое вино: ...","Калорий 785 ккал, Белки 54 грамм, Жиры 16 грам...",Итальянская кухня,50 минут,Ризотто
20100,Ризотто со шпинатом и пармезаном,Растопите в сковороде половину сливочного масл...,"Рис карнароли: 200 г, Шпинат: 200 г, Сливочное...","Калорий 342 ккал, Белки 9 грамм, Жиры 17 грамм...",Итальянская кухня,40 минут,Ризотто
20127,Ризотто с грушей и сыром пекорино,"Очистите грушу, удалите сердцевину. Отложите н...","Груши: 1 штука, Лимонный сок: 1 столовая ложка...","Калорий 610 ккал, Белки 11 грамм, Жиры 38 грам...",Итальянская кухня,30 минут,Ризотто
20022,Ризотто с редиской,В глубоком и широком сотейнике растопить сливо...,"Редис: 100 г, Рис арборио: 400 г, Чеснок: 2 зу...","Калорий 655 ккал, Белки 19 грамм, Жиры 26 грам...",Итальянская кухня,30 минут,Ризотто


Пятая рекомендация

In [29]:
idx = np.random.randint(df_items.shape[0])
text_input = "Блинчики с вараньем"
print(text_input)
text_input = preproccesing(text_input, tokenizer)

Блинчики с вараньем


In [30]:
rating = get_recommendation(text_input, model, items_embed, df_items)
df_items.iloc[rating].head()

,name,text,ingredient,energy,time_cook,type_kitchen,label
1233,Блинчики с плавленным сыром,"Блинчики можно купить уже готовые, а можно вып...","Тонкие блины : 3 штуки, Плавленый сыр: 6 штук,...","Калорий 3197 ккал, Белки 205 грамм, Жиры 142 г...",Русская кухня,5 минут,Завтрак
1082,Блинчики из отрубей по Дюкану,Отруби смолоть в кофемолке или блендере в муку...,"Овсяные отруби: 2 столовые ложки, Пшеничные от...","Калорий 182 ккал, Белки 14 грамм, Жиры 8 грамм...",Европейская кухня,25 минут,Завтрак
29152,Блинчики из отрубей,"Смешать отруби, творог и яйцо (или только его ...","Отруби: 60 г, Мягкий обезжиренный творог: 1,5 ...","Калорий 187 ккал, Белки 19 грамм, Жиры 9 грамм...",Русская кухня,10 минут,Выпечка
34454,Яичные блинчики с креветками,"Креветки промыть, добавить нашинкованный лук, ...","Очищенные креветки: 100 г, Куриное яйцо: 4 шту...","Калорий 274 ккал, Белки 10 грамм, Жиры 20 грам...",Блины,35 минут,Выпечка
1131,"Яичные блинчики с помидорами, зеленью, чесноко...","Яйца, молоко, зелень, соль взбиваем и выпекаем...","Куриное яйцо: 1 штука, Молоко: 30 мл, Соль: по...","Калорий 282 ккал, Белки 11 грамм, Жиры 24 грам...",Русская кухня,25 минут,Завтрак
